### 데이터 및 라이브러리 불러오기

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811606 sha256=bb95b8c5f79d425adc4c9eb42c7ff31b000053414fe9ab26b94edadfe1b58001
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from surprise import Reader, Dataset
import math

In [ ]:
# 경고 무시
pd.set_option('mode.chained_assignment',  None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
book = pd.read_csv('/content/drive/MyDrive/2023/문화데이터경진대회/final/추천도서목록.csv')
movie = pd.read_csv('/content/drive/MyDrive/2023/문화데이터경진대회/final/영화목록.csv')
book_rating = pd.read_csv('/content/drive/MyDrive/2023/문화데이터경진대회/final/사용자_도서_평점.csv')
movie_rating = pd.read_csv('/content/drive/MyDrive/2023/문화데이터경진대회/final/사용자_영화_평점.csv')

### 책 데이터

In [ ]:
import re
book['책소개 내용'] = book['책 소개'].map(lambda x: x.replace('\n',' '))

In [ ]:
book

,상품명,인물,책 소개,책소개_전처리,책소개 내용
0,만일 내가 인생을 다시 산다면,김혜남,“하나의 문이 닫히면 또 하나의 문이 열린다. \n그러니 더 이상 고민하지 말고 그...,이상 고민 정신 분석 전문의 김혜남 마흔 당신 그녀 가지 후회 스스로 닦달 인생 숙...,“하나의 문이 닫히면 또 하나의 문이 열린다. 그러니 더 이상 고민하지 말고 그냥...
1,원씽 The One Thing,게리 켈러 외,전 세계에서 두 번째로 큰 투자개발 회사의 대표이자 전미 130만 부 이상이 팔린 ...,세계 투자 개발 회사 대표 전미 이상 게리 캘러 통념 개념 기계 인생 성공 행복 진...,전 세계에서 두 번째로 큰 투자개발 회사의 대표이자 전미 130만 부 이상이 팔린 ...
2,불편한 편의점,김호연,"원 플러스 원의 기쁨, 삼각김밥 모양의 슬픔, 만 원에 네 번의 폭소가 터지는 곳!...",플러스 기쁨 삼각김밥 슬픔 폭소 시대 조금 특별 편의점 사랑 소설 서점 종합 베스트...,"원 플러스 원의 기쁨, 삼각김밥 모양의 슬픔, 만 원에 네 번의 폭소가 터지는 곳!..."
3,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키,"제26회 전격소설대상 ‘미디어워크스문고상’ 수상작\n총 4,607편의 응모 작품 중...",미디어 워크스 문고 상작 눈물 기억 리셋 선행 기억 실증 소녀 히노 마오리 무미건조...,"제26회 전격소설대상 ‘미디어워크스문고상’ 수상작 총 4,607편의 응모 작품 중 ..."
4,아버지의 해방일지,정지아,"새삼스럽게 경탄스럽다!\n압도적인 몰입감, 가슴 먹먹한 감동\n정지아의 손끝에서 펼...",경탄 입감 가슴 감동 정지아 시대 온기 미스터리 남자 역사 격랑 발견 인생 김유정 ...,"새삼스럽게 경탄스럽다! 압도적인 몰입감, 가슴 먹먹한 감동 정지아의 손끝에서 펼쳐지..."
...,...,...,...,...,...
1312,소마,채사장,"※양장본 리커버 에디션: 매장 구매, 바로드림 구매 시에는 일반판 랜덤으로 제공됩니...",일반 한국 인문학 스토리텔링 지식 천일야화 사장 소설 한국 사회 도착 서사 사장 등...,"※양장본 리커버 에디션: 매장 구매, 바로드림 구매 시에는 일반판 랜덤으로 제공됩니..."
1313,얼음 속의 엄마를 떠나보내다,남유하,한낙원과학소설상 수상ㆍ과학소재공모전 우수상에 수상 빛나는 작가. 남유하의 판타지 고...,한낙원 과학 소설 유하 판타지 고딕 호러 소설 북쪽 전설 겨울 계속 마을 마을 부족...,한낙원과학소설상 수상ㆍ과학소재공모전 우수상에 수상 빛나는 작가. 남유하의 판타지 고...
1314,헌책방 기담 수집가,윤성근,사연 있는 책을 찾아드립니다 \n수수료는 당신 삶의 이야기!\n\n감동과 미스터리가...,사연 수수료 당신 감동 미스터리 실화 속편 장강명 소설가 누군가 책방 계단 주인 인...,사연 있는 책을 찾아드립니다 수수료는 당신 삶의 이야기! 감동과 미스터리가 어우...
1315,세상 끝 아파트에서 유령을 만나는 법,정지윤,"증강현실이 간판과 길거리 공연, 실시간 뉴스까지 눈앞에 바로 제공해주는 확장현실로 ...",증강현실 간판 길거리 공연 실시간 뉴스 눈앞 확장 현실 발달 미래 서울 확장 현실 ...,"증강현실이 간판과 길거리 공연, 실시간 뉴스까지 눈앞에 바로 제공해주는 확장현실로 ..."


In [ ]:
# 책 삭제
book.drop([1283], inplace=True) # 한명숙의 진실

In [ ]:
book.reset_index(drop=True, inplace=True)

### 영화 데이터

In [ ]:
movie

,영화명,줄거리,장르,키워드,줄거리_전처리,줄거리+키워드,영화명_전처리
0,명량,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...","시대극/사극,액션,전기",사극 수군 실화바탕 실화바탕 역사물 위인 이순신 임진왜란 전쟁 해전 명량해전 명량대...,임진왜란 전쟁 혼란 조선 속도 한양 북상 왜군 국가 존망 위기 누명 이순신 장군 최...,임진왜란 전쟁 혼란 조선 속도 한양 북상 왜군 국가 존망 위기 누명 이순신 장군 최...,명량
1,극한직업,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,코메디,잠복근무 마약단속반 위장경찰 위장취업 형사 경찰 마약범죄단 마약밀매업자 마약상 맛집...,장사 잠복근무 지금 수사 불철주야 실적 바닥 해체 위기 마약 이상 맏형 반장 국제 ...,장사 잠복근무 지금 수사 불철주야 실적 바닥 해체 위기 마약 이상 맏형 반장 국제 ...,극한직업
2,신과함께-죄와 벌,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인...","드라마,판타지",웹툰원작 저승 저승사자 검사 사후세계 시리즈 소방관 염라대왕,저승 인간 사후 재판 살인 나태 거짓 배신 폭력 천륜 지옥 재판 통과 망자 환생 시...,저승 인간 사후 재판 살인 나태 거짓 배신 폭력 천륜 지옥 재판 통과 망자 환생 시...,신과함께-죄와벌
3,국제시장,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,드라마,6.25 6.25 625 현대사 한국전쟁 부산 부산광역시,아버지 한국 전쟁 현재 격변 시대 관통 시대 아버지 덕수 황정민 평생 자신 다행 눈...,아버지 한국 전쟁 현재 격변 시대 관통 시대 아버지 덕수 황정민 평생 자신 다행 눈...,국제시장
4,어벤져스: 엔드게임,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,"액션,SF",마블코믹스 슈퍼히어로 다크히어로 어벤져스 아이언맨 캡틴아메리카 블랙위도우 토르 헐크...,인피니티 절반 지구 마지막 희망 어벤져스 어벤져스 운명 최후 전쟁,인피니티 절반 지구 마지막 희망 어벤져스 어벤져스 운명 최후 전쟁마블코믹스 슈퍼히어...,어벤져스:엔드게임
...,...,...,...,...,...,...,...
492,오늘의 연애,해 달라는 대로 다 해주는데 100일도 못 가 항상 여자친구에게 차이는 초등학교 교...,"드라마,멜로드라마",사랑 연애 친구 소꿉친구 기상캐스터 썸,여자 친구 차이 초등 학교 교사 준수 이승기 여자 마음 숙제 준수 이승기 특별 썸녀...,여자 친구 차이 초등 학교 교사 준수 이승기 여자 마음 숙제 준수 이승기 특별 썸녀...,오늘의연애
493,글러브,퇴출직전의 꼴통 프로 투수와 말도 안 통하고(?) 꽉 막힌 만년꼴찌 야구부원들의 1...,"드라마,스포츠",고교야구 도전 스포츠 실화바탕 실화바탕 야구 야구감독 야구부 야구선수 야구스타 전국...,퇴출 꼴통 프로 투수 만년 꼴찌 야구 부원 웃음 감동 전기 소리 파이팅 글러브 처음...,퇴출 꼴통 프로 투수 만년 꼴찌 야구 부원 웃음 감동 전기 소리 파이팅 글러브 처음...,글러브
494,러브 액츄얼리,새로 부임한 매력적인 미혼의 영국 수상(휴 그랜트)은 발랄하고 귀여운 비서 나탈리(...,"코메디,드라마,멜로/로맨스",결혼 공항 로맨스 수상 짝사랑 크리스마스 재개봉 옴니버스형식,부임 매력 미혼 영국 그랜트 비서 나탈리 마틴 맥커친 첫눈 자신 위치 의식 그녀 매...,부임 매력 미혼 영국 그랜트 비서 나탈리 마틴 맥커친 첫눈 자신 위치 의식 그녀 매...,러브액츄얼리
495,라디오 스타,"명곡 비와 당신으로 88년 가수 왕을 차지했던 최곤은 그 후 대마초 사건, 폭행사건...","코메디,드라마",대마초 인기가수 통기타 매니저 라디오 라디오DJ 라디오쇼 라디오PD 다방 다방레지 ...,명곡 당신 가수 차지 최곤 대마초 사건 폭행 사건 연루 커플 상대 미사리 까페 기타...,명곡 당신 가수 차지 최곤 대마초 사건 폭행 사건 연루 커플 상대 미사리 까페 기타...,라디오스타


In [ ]:
# 영화명 수정
movie.loc[93,'영화명'] = '가문의 위기 - 가문의 영광 2'
movie.loc[124,'영화명'] = '캐리비안의 해적 - 세상의 끝에서'
movie.loc[146,'영화명'] = '캐리비안의 해적 - 망자의 함'
movie.loc[461,'영화명'] = '캐리비안의 해적 - 블랙 펄의 저주'
movie.loc[165,'영화명'] = '인디아나 존스: 크리스탈 해골의 왕국'
movie.loc[227,'영화명'] = '다이 하드 4.0'
movie.loc[317,'영화명'] = '적벽대전 2부 - 최후의 결전'
movie.loc[319,'영화명'] = '지.아이.조 - 전쟁의 서막'

In [ ]:
m_lst = [93,124,146,461,165,227,317,319]
movie.loc[m_lst,'영화명_전처리'] = movie.loc[m_lst,'영화명'].map(lambda x: x.replace(' ',''))

### 책 평점 데이터

In [ ]:
book_rating

,user,book,book_ID,rating
0,DgwxAeQYNxrMj,물고기는 존재하지 않는다,byLKj8M,5.0
1,DgwxAeQYNxrMj,21세기를 위한 21가지 제언,bo50Jrl,4.0
2,DgwxAeQYNxrMj,파친코,b9RlDAY,3.5
3,DgwxAeQYNxrMj,소크라테스 익스프레스,bo5qAzM,3.5
4,DgwxAeQYNxrMj,미드나잇 라이브러리,byzQq6k,2.0
...,...,...,...,...
66612,RE952nyEZ5Q72,자기만의 방,bovdKxP,3.5
66613,2mwvgaoMgxMa7,자기만의 방,bovdKxP,4.5
66614,WwRqoggDDqlzB,자기만의 방,bovdKxP,4.5
66615,g64qzYYdWxER0,자기만의 방,bovdKxP,3.5


In [ ]:
# 책 평점 삭제
idx1 = book_rating[book_rating['book'].map(lambda x: '한명숙' in x)].index
book_rating.drop(idx1, inplace=True)
book_rating.reset_index(drop=True, inplace=True)

### 영화 평점 데이터

In [ ]:
# rating == '보고싶어요' 인 평점 삭제
drop_movie = movie_rating[movie_rating['rating'].map(lambda x: x == '보고싶어요')]
movie_rating.drop(drop_movie.index, inplace=True)
movie_rating

,user,movie,movie_ID,rating
0,Pznx9j3wJq761,명량,mWzM9D5,2.0
1,8nPvyyWmovYo0,명량,mWzM9D5,1.0
2,DgwxAeQYNxrMj,명량,mWzM9D5,2.5
3,zNM5NrKAQ526j,명량,mWzM9D5,2.0
4,JgAx8QgkOqLbO,명량,mWzM9D5,1.5
...,...,...,...,...
27554,JgAx89MRDqLbO,블라인드,m5e77oO,3.0
27555,nkPvrQAPJqgar,블라인드,m5e77oO,3.0
27556,a9L5PwKN2vwg8,블라인드,m5e77oO,3.0
27557,24BqEMAnOqrjn,블라인드,m5e77oO,3.0


In [ ]:
# 영화명 띄어쓰기 없애기
movie_rating['movie'] = movie_rating['movie'].map(lambda x: x.replace(' ',''))

In [ ]:
movie_pivot = movie_rating.pivot(index='user', columns='movie', values='rating')
movie_pivot = movie_pivot.fillna(0)
movie_pivot

movie,#살아있다,007스카이폴,007퀀텀오브솔러스,1987,1번가의기적,2012,26년,300,7광구,7급공무원,...,화려한휴가,화이:괴물을삼킨아이,화차,황금나침반,황산벌,황해,효자동이발사,후궁:제왕의첩,히말라야,히트맨
user,,,,,,,,,,,,,,,,,,,,,
06nkPvrlxgarB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0LzNM5Nox26j7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17ovV08glxzyn,0,4.0,3.5,0,0,0,0.5,0,0,0,...,0,0,0,3.0,0,0,0,0,0,0
17ovV0Br1xzyn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17ovV0JrLxzyn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zNM5NzQDn526j,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zNM5NzXBQ526j,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zNM5NzbLo526j,0,0,0,0,0.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 줄거리 기반 도서 추천 시스템

In [ ]:
def content_movie_book_recommendation():
    movie_like = input('🎬 좋아하는 영화의 제목을 입력하세요 (여러 개는 쉼표로 구분해서)\n: ')
    movie_like = movie_like.replace(' ','')
    movie_like = movie_like.split(',')

    movie_sel = movie[movie['영화명_전처리'].map(lambda x: any(mov in x for mov in movie_like))]
    movie_list = list(movie_sel['영화명'].values)
    print('\n')
    print('🎞️ 선택된 영화')
    print(':', movie_sel['영화명'].values)

    movie_word = movie_sel['줄거리+키워드'].values.tolist()
    movie_word = ' '.join(movie_word)

    movie_book = pd.DataFrame(book['책소개_전처리'])
    movie_book.loc[1316] = movie_word

    vectorizer = TfidfVectorizer()
    features = vectorizer.fit_transform(movie_book['책소개_전처리'])
    feature_names = vectorizer.get_feature_names_out()
    dtm = np.array(features.todense())

    cosine_sim = cosine_similarity(dtm, dtm)
    sim = pd.DataFrame(cosine_sim)
    book_reco = list(enumerate(sim[1316]))
    book_reco = sorted(book_reco, key=lambda x: x[1], reverse=True)
    book_idx = book_reco[1:11]
    book_sim = [i[1] for i in book_idx]
    book_idx = [i[0] for i in book_idx]

    books = book['상품명'].iloc[book_idx]

    print('\n')
    print('📖 {}를 좋아하는 사용자에게 추천하는 상위 10개 도서\n'.format(movie_list))
    for i in books:
        print('▪️ {}'.format(i))
        print(': {} ....더보기'.format(book[book['상품명']==i]['책소개 내용'].values[0][:140]))
        print('\n')

In [ ]:
content_movie_book_recommendation()

🎬 좋아하는 영화의 제목을 입력하세요 (여러 개는 쉼표로 구분해서)
: 해리포터


🎞️ 선택된 영화
: ['해리포터와 죽음의 성물2' '해리포터와 비밀의 방' '해리 포터와 불의 잔' '해리 포터와 불사조 기사단'
 '해리 포터와 혼혈 왕자' '해리 포터와 죽음의 성물1' '해리포터와 아즈카반의 죄수']


📖 ['해리포터와 죽음의 성물2', '해리포터와 비밀의 방', '해리 포터와 불의 잔', '해리 포터와 불사조 기사단', '해리 포터와 혼혈 왕자', '해리 포터와 죽음의 성물1', '해리포터와 아즈카반의 죄수']를 좋아하는 사용자에게 추천하는 상위 10개 도서

▪️ 해리 포터와 저주받은 아이
: 19년 뒤의 킹스크로스역에서 다시 시작되는 이야기!  J.K. 롤링과 잭 손, 존 티퍼니가 쓴 원작을 토대로 잭 손이 각색하여 탄생시킨 희곡 『해리포터와 저주받은 아이』 제1부. 79개 국어로 번역돼 4억 5,000만 부가 팔리며 유례를 찾기 힘들 만 ....더보기


▪️ 해리 포터와 비밀의 방
: 선과 악의 대립 속에서 평범한 어린 소년이 한 사람의 영웅으로 성장해나가는 보편적인 테마를 바탕으로 빈틈없는 소설적 구성과 생생하게 살아 있는 캐릭터, 정교하게 만들어낸 환상의 세계를 접목시킨 21세기의 고전 「해리포터 시리즈」의 제2편『해리포터와 비 ....더보기


▪️ 해리 포터와 마법사의 돌
: 양장으로 다시 만나는 해리포터 시리즈!  선과 악의 대립 속에서 평범한 어린 소년이 한 사람의 영웅으로 성장해나가는 보편적인 테마를 바탕으로 빈틈없는 소설적 구성과 생생하게 살아 있는 캐릭터, 정교하게 만들어낸 환상의 세계를 접목시킨 21세기의 고전  ....더보기


▪️ 바퀴벌레
: 바퀴벌레처럼 사방에 존재하지만 애써 못 본 척해온 진실을 향해 다가서는 해리 홀레!  인기 뮤지션, 저널리스트, 경제학자로도 활동하고 있는 작가에게 명성을 안겨준 「해리 홀레」 시리즈의 두 번째 소설 『바퀴벌레』. 전작 《박쥐》와 함께 해리 홀레의 청 ....더보기


▪️ 위니를 

### 선호영화 기반 도서 추천 시스템, '마음의 냥식'

In [ ]:
def hybrid_movie_book_recommendation():
    try:
        movie_like = input('🎬 좋아하는 영화의 제목을 입력하세요 (여러 개는 쉼표로 구분해서)\n: ')
        movie_like = movie_like.replace(' ','')
        movie_like = movie_like.split(',')

        movie_sel = movie[movie['영화명_전처리'].map(lambda x: any(mov in x for mov in movie_like))]
        print('\n')
        print('🎞️ 선택된 영화')
        print(':', movie_sel['영화명'].values)

        movie_word = movie_sel['줄거리+키워드'].values.tolist()
        movie_word = ' '.join(movie_word)

        movie_book = pd.DataFrame(book['책소개_전처리'])
        movie_book.loc[1316] = movie_word

        vectorizer = TfidfVectorizer()
        features = vectorizer.fit_transform(movie_book['책소개_전처리'])
        dtm = np.array(features.todense())

        cosine_sim = cosine_similarity(dtm, dtm)
        sim = pd.DataFrame(cosine_sim)
        book_reco = list(sim[1316])

        book_weight = pd.DataFrame(index=['book','similarity'], data=[book['상품명'].values, book_reco[:-1]]).T

    #  -----------------------------------------------------------------------------------

        movie_list = list(movie_sel['영화명'].values)
        movie_list_pro = list(movie_sel['영화명_전처리'].values)
        movie_like_rating = movie_pivot[movie_list_pro]
        movie_like_rating.loc['user'] = 5.0

        for i in movie_list_pro:
            movie_like_rating[i] = movie_like_rating[i].map(lambda x: float(x))

        se = []
        for m in movie_list_pro:
            se.append(list(movie_like_rating[m].map(lambda x: (5-x)**2).values))
        sim = np.sum(se, axis=0)
        sim = list(enumerate(sim))
        sim_score = sorted(sim, key=lambda x : x[1])
        sim_idx = [i[0] for i in sim_score[1:101]]
        sim_user = list(movie_like_rating.iloc[sim_idx].index)

        user_book = book_rating[book_rating['user'].map(lambda x: x in sim_user)]
        book_lst = list(user_book['book'].unique())

        book_final = user_book.groupby('book').mean().reset_index()

        book_weight = book_weight[book_weight['book'].map(lambda x: x in book_lst)]
        book_cat = pd.merge(book_final,book_weight,on='book')

        alpha = 0.2
        beta = 10
        book_cat['weighted_rating'] = (alpha * book_cat['rating']) + (beta * book_cat['similarity'])
        book_cat = book_cat.sort_values('weighted_rating', ascending=False)
        books = book_cat[:10]['book']

        print('\n')
        print('📖 {}를 좋아하는 사용자에게 추천하는 상위 10개 도서\n'.format(movie_list))
        for i in books:
            print('▪️ {}'.format(i))
            print(': {} ....더보기'.format(book[book['상품명']==i]['책소개 내용'].values[0][:140]))
            print('\n')

    except:
        print('\n')
        print('😥 죄송합니다. 선택된 영화가 없어 도서를 추천할 수 없습니다.')
        print('영화의 제목을 다시 한 번 확인해주시거나, 다른 영화를 입력해 주십시오.')

In [ ]:
hybrid_movie_book_recommendation()

🎬 좋아하는 영화의 제목을 입력하세요 (여러 개는 쉼표로 구분해서)
: 인턴


🎞️ 선택된 영화
: ['인턴']


📖 ['인턴']를 좋아하는 사용자에게 추천하는 상위 10개 도서

▪️ 도시는 무엇으로 사는가
: 도시는 그 안에 사는 사람들을 닮는다!  도시는 도시 계획을 한 디자이너의 손을 떠나는 순간 이내 진화를 시작한다. 그 안에 사는 사람들의 삶이 반영되기 때문에 인간이 추구하는 것과 욕망이 드러난다. 하다못해 작은 사무실의 상사와 부하 직원의 자리배치 ....더보기


▪️ 원씽 The One Thing
: 전 세계에서 두 번째로 큰 투자개발 회사의 대표이자 전미 130만 부 이상이 팔린 베스트셀러의 저자 게리 캘러가 쓴 이제까지의 통념을 뒤엎는 신개념 자기계발서이다. 이 책이 말하고 있는 인생의 성공과 행복에 대한 진리는 너무도 단순해서 김이 빠질 지경 ....더보기


▪️ 인간 본성의 법칙
: 500만 년에 걸쳐 형성된 인간 본성에 관한 탐구서!  《권력의 법칙》의 저자 로버트 그린이 우리 안에 숨겨진 인간 본성에 관한 18가지 법칙을 통찰해내며 평범하고, 이상하고, 파괴적인 모습이 공존하는 매혹 될 수밖에 없는 존재, 인간의 진짜 모습을  ....더보기


▪️ 그릿 GRIT
: 끝나지 않는 역주행 돌풍!  켈리 최, 자청, 신사임당…  셀럽, 리더, 언론이 추천한 새로운 자기계발서의 고전!  국내 50만 부 돌파, 150쇄 기념 《그릿 GRIT》 골드 에디션 출간! “당신에겐 ‘그릿’이 있는가?” 실패, 역경, 슬럼프를 이겨 ....더보기


▪️ 설득의 심리학 1
: 초판 출간 이후 37년 동안 44여 개 언어로 번역되며,  전 세계 누적 판매 500만 부를 돌파한 ‘비즈니스의 고전’  변화를 읽고 마음을 끌어당기는 우리 시대 최고의 자기계발서  『설득의 심리학』이 20주년 완전 개정판으로 돌아왔다!  세계적으로  ....더보기


▪️ 자기만의 방
: ‘지금 이곳’에 꼭 필요한 책을 만나다!  1966년 창립된 출판사 민음사의 로고 ‘

<ipython-input-20-87a40759675e>:50: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  book_final = user_book.groupby('book').mean().reset_index()
